In [6]:
import numpy as np
import pandas as pd
from math import exp
from sklearn.model_selection import train_test_split
from collections import Counter
from imblearn.over_sampling import RandomOverSampler
from sklearn.metrics import matthews_corrcoef

In [38]:
def train_data():
    df = pd.read_csv('train.dat', delimiter='\t', names=["train_labels","train_sequence"])
    return df

In [8]:
def test_data():
    df = pd.read_csv('test.dat', delimiter='\t', names=["test_sequence"])
    return df

In [39]:
def extract_kmers_and_features(X, k=4):
    """
    Extract k-mers for each sequence in X and return the unique k-mers as features.
    """
    def kmer(seq, size):
        return [seq[x:x+size].lower() for x in range(0, len(seq) - size + 1)]

    # Extract k-mers for each sequence in X
    X_kmers = []
    all_kmers = set()
    for seq in X:
        kmers_seq = []
        for size in range(1, k):  # Generate k-mers from size 1 to k-1
            kmers_seq.extend(kmer(seq, size))
        X_kmers.append(' '.join(kmers_seq))
        all_kmers.update(kmers_seq)

    # Return k-mer sequences and sorted unique k-mers (feature set)
    return X_kmers, sorted(list(all_kmers))


In [40]:
df = train_data()
X, Y = df['train_sequence'].to_numpy(), df['train_labels'].to_numpy()
X_kmers, features = extract_kmers_and_features(X, k=4)
print(X_kmers)

['d v e l d l v e i s p n a l p dv ve el ld dl lv ve ei is sp pn na al lp dve vel eld ldl dlv lve vei eis isp spn pna nal alp', 'k a d e e l f n k l f f g t ka ad de ee el lf fn nk kl lf ff fg gt kad ade dee eel elf lfn fnk nkl klf lff ffg fgt', 'f l v a l h l g t a f a l l w y f r k r w c a l v r g f f a s f g g r r n d d a h m m fl lv va al lh hl lg gt ta af fa al ll lw wy yf fr rk kr rw wc ca al lv vr rg gf ff fa as sf fg gg gr rr rn nd dd da ah hm mm flv lva val alh lhl hlg lgt gta taf afa fal all llw lwy wyf yfr frk rkr krw rwc wca cal alv lvr vrg rgf gff ffa fas asf sfg fgg ggr grr rrn rnd ndd dda dah ahm hmm', 'r d q m r a r i a d i t g v a i s r i a rd dq qm mr ra ar ri ia ad di it tg gv va ai is sr ri ia rdq dqm qmr mra rar ari ria iad adi dit itg tgv gva vai ais isr sri ria', 'r k r l q l l l l rk kr rl lq ql ll ll ll rkr krl rlq lql qll lll lll', 'p g f c v g e a s p l k s p g r r e l g h g n l a pg gf fc cv vg ge ea as sp pl lk ks sp pg gr rr re el lg gh hg gn nl la pgf gfc

In [41]:
def vectorize_kmers(X_kmers, features):
    """
    Convert k-mer sequences into vector representation based on features.
    """
    X_vector = np.zeros((len(X_kmers), len(features)))

    for i, kmers_seq in enumerate(X_kmers):
        count = Counter(kmers_seq.split())
        for kmer, cnt in count.items():
            if kmer in features:
                X_vector[i][features.index(kmer)] = cnt
        # Normalize the vector by dividing by the maximum value in the vector
        if np.max(X_vector[i]) > 0:
            X_vector[i] /= np.max(X_vector[i])

    return X_vector

In [42]:
X_vector = vectorize_kmers(X_kmers, features)

In [43]:
print(X_vector[0])

[0.33333333 0.         0.         ... 0.         0.         0.        ]


In [44]:
print(X_vector.shape)

(1566, 7264)


In [15]:
def oversample(X, Y):
    oversample = RandomOverSampler(sampling_strategy=0.2, random_state = 42)
    X_over, Y_over = oversample.fit_resample(X, Y)
    return X_over, Y_over

In [16]:
def relu(x):
    return np.maximum(0.0, x)

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def forward_prop(X, W1, W2, W3, b1, b2, b3):

    z1 = relu(np.dot(X,W1) + b1)
    z2 = relu(np.dot(z1, W2) + b2)
    y_pred = sigmoid(np.dot(z2, W3) + b3)

    return z1, z2, y_pred
batch_size=128
neuronsl1, neuronsl2 =600 , 100
lambda_reg=0.0001
lr = 0.001

In [17]:
def compute_loss(y_pred, y, W1, W2, W3):
    return np.sum(((y-y_pred) ** 2) + lambda_reg * (np.linalg.norm(W1)+np.linalg.norm(W2)+np.linalg.norm(W3)))

In [18]:
def sigmoid_derivative(s_x):
    return np.multiply(s_x, np.subtract(1.0, s_x))

def relu_derivative(x):
    return np.where(x>0, 1, 0)

def backward_prop(X, Y, y_pred, Z1, Z2, W1, W2, W3, b1, b2, b3):
    delta_w3 = sigmoid_derivative(y_pred) * (Y-y_pred) #number of instances * number of output neurons
    delta_w2 = relu_derivative(Z2) * np.dot(delta_w3, W3.T)
    delta_w1 = relu_derivative(Z1) * np.dot(delta_w2, W2.T)

    W3 = W3 + (lr * (np.dot(Z2.T, delta_w3)))
    W3 = W3 - (lr * lambda_reg * np.sign(W3))
    b3 = b3 + (lr * np.dot(np.ones((1,delta_w3.shape[0])), delta_w3))
    W2 = W2 + (lr * (np.dot(Z1.T, delta_w2)))
    W2 = W2 - (lr * lambda_reg * np.sign(W2))
    b2 = b2 + (lr * np.dot(np.ones((1,delta_w2.shape[0])), delta_w2))
    W1 = W1 + (lr * (np.dot(X.T, delta_w1)))
    W1 = W1 - (lr * lambda_reg * np.sign(W1))
    b1 = b1 + (lr * np.dot(np.ones((1,delta_w1.shape[0])), delta_w1))
    return W1, W2, W3, b1, b2, b3

In [19]:
def train_fnn(X_train, y_train, features, epochs):
    # Initialize weights and biases
    b1 = np.zeros((1, neuronsl1))
    b2 = np.zeros((1, neuronsl2))
    b3 = np.zeros((1))
    R = np.random.RandomState(2025)
    W1 = R.normal(0, (2 / X_train.shape[1]), (len(features), neuronsl1))
    W2 = R.normal(0, (2 / neuronsl1), (neuronsl1, neuronsl2))
    W3 = R.normal(0, (2 / neuronsl2 + 1), (neuronsl2, 1))

    y_train = y_train[np.newaxis].T
    training_loss = []

    for epoch in range(epochs):
        # Training loop
        for i in range(0, len(X_train), batch_size):
            Xbatch = X_train[i:i + batch_size]
            ybatch = y_train[i:i + batch_size]
            Z1, Z2, y_pred = forward_prop(Xbatch, W1, W2, W3, b1, b2, b3)
            W1, W2, W3, b1, b2, b3 = backward_prop(Xbatch, ybatch, y_pred, Z1, Z2, W1, W2, W3, b1, b2, b3)

        # Calculate training loss
        Z1, Z2, y_pred = forward_prop(X_train, W1, W2, W3, b1, b2, b3)
        loss = compute_loss(y_pred, y_train, W1, W2, W3)
        training_loss.append(loss)
        print(f'Epoch {epoch}: Training Loss = {loss}')

    return W1, W2, W3, b1, b2, b3, training_loss


In [20]:
X_train, X_test, y_train, y_test = train_test_split(X_vector,
                                                    Y,
                                                    test_size = 0.20,
                                          random_state=42, stratify=Y)

X_over, Y_over = oversample(X_train, y_train)

Y_over = np.where(Y_over<0, 0, 1)
y_test = np.where(y_test<0, 0, 1)

W1, W2, W3, b1, b2, b3, training_loss = train_fnn(X_over, Y_over, features, epochs =200)

Epoch 0: Training Loss = 199.81349646576888
Epoch 1: Training Loss = 194.77377825017473
Epoch 2: Training Loss = 193.81555098015158
Epoch 3: Training Loss = 193.33511426336986
Epoch 4: Training Loss = 192.85838058539937
Epoch 5: Training Loss = 192.25595592160698
Epoch 6: Training Loss = 191.46051525961138
Epoch 7: Training Loss = 190.4002526291585
Epoch 8: Training Loss = 188.99206729993952
Epoch 9: Training Loss = 187.1445813257156
Epoch 10: Training Loss = 184.77314030849936
Epoch 11: Training Loss = 181.86791388844753
Epoch 12: Training Loss = 178.55316967456307
Epoch 13: Training Loss = 175.99003633860403
Epoch 14: Training Loss = 175.79602922377114
Epoch 15: Training Loss = 174.47893044929006
Epoch 16: Training Loss = 157.90320768345208
Epoch 17: Training Loss = 131.76050988807546
Epoch 18: Training Loss = 120.00603471337506
Epoch 19: Training Loss = 110.69958294305113
Epoch 20: Training Loss = 103.78798434290958
Epoch 21: Training Loss = 98.93887720720882
Epoch 22: Training Loss

In [21]:
def predict(X_test, y_test, W1, W2, W3, b1, b2, b3):
    # Predict on the validation set
    Z1, Z2, y_pred = forward_prop(X_test, W1, W2, W3, b1, b2, b3)
    y_pred = np.where(y_pred < 0.6, 0, 1)
    validation_mcc = matthews_corrcoef(y_pred, y_test[np.newaxis].T)
    print(f'Validation MCC = {validation_mcc}')
    return y_pred, validation_mcc

In [22]:
y_pred, validation_mcc = predict(X_test, y_test, W1, W2, W3, b1, b2, b3)

Validation MCC = 0.9215784215784216


In [45]:
df = test_data()
X_t = df['test_sequence'].to_numpy()
X_kmers_t, features_t = extract_kmers_and_features(X_t, k=4)
X_vector_t = vectorize_kmers(X_kmers_t, features)


In [49]:
Z1, Z2, y_pred = forward_prop(X_vector, W1, W2, W3, b1, b2, b3)
y_pred = np.where(y < 0.6, -1, 1)
np.savetxt('predictions.dat', y, delimiter='\n',fmt='%i')

In [47]:
print(X_vector_t.shape)

(392, 7264)
